In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler,RobustScaler

In [ ]:
def data_normalization(train_set, test_set, fn, projectid):
    print('fn: ',fn)
    if fn == 'MM':
        scaler = MinMaxScaler()
    elif fn == 'Z':
        scaler = StandardScaler()
    elif fn == 'MA':
        scaler = MaxAbsScaler()
    elif fn == 'R':
        scaler = RobustScaler()
    classes = None
    X_test_scaled = []
    validation_label = None
    if projectid.split("-")[0][0] == 'B' or projectid.split("-")[0][0] == 'M':
        data, label = classification_process(train_set)
        label, classes = label_pre(label)
        validation_data = []  # 预先定义
        ifval = False
        if len(test_set) > 0:
            validation_data, validation_label = classification_process(test_set)
            validation_label, classes = label_pre(validation_label)
            ifval = True
    elif projectid.split("-")[0][0] == 'R':
        data, label = regression_preprocess(train_set)
        ifval = False
        if len(test_set) > 0:
            validation_data, validation_label = regression_preprocess(test_set)
            ifval = True
    elif projectid.split("-")[0][0] == 'S':
        data, label = sur_data_process(train_set)
        ifval = False
        if len(test_set) > 0:
            validation_data, validation_label = sur_data_process(test_set)
            ifval = True

    X_train_scaled = scaler.fit_transform(data)
    X_train_scaled = pd.DataFrame(X_train_scaled,index=data.index,columns=data.columns)

    if len(test_set) > 0:
        X_test_scaled = scaler.transform(validation_data)
        X_test_scaled = pd.DataFrame(X_test_scaled, index=validation_data.index, columns=validation_data.columns)

    norm_data = {'train_set': X_train_scaled,
                 'test_set': X_test_scaled,
                 'train_set_label': label,
                 'test_set_label': validation_label,
                 'scaler': scaler,
                 'classes': classes,
                 'ifval': ifval
                 }
    print('normalization!')
    return norm_data

In [ ]:
# preprocessing
def classification_process(data):
    data = data.apply(pd.to_numeric,errors='ignore') #转成数值型
    x=data.iloc[:,data.columns!=data.columns[0]]
    if np.any(x.isnull()) == True:
        x=x.fillna(x.mean())  #填充缺失值
    y=np.array(data.iloc[:,0]).ravel()
    return x,y

def regression_preprocess(data):
    data = data.apply(pd.to_numeric, errors='ignore')
    x = data.iloc[:, data.columns != data.columns[0]]
    x = x.fillna(x.mean())  # 填充缺失值
    x_dum = pd.get_dummies(x)  # 独热编码
    y = np.array(data.iloc[:, 0]).ravel()
    return x_dum, y

def sur_data_process(data):
    data = data.rename(columns={data.columns[0]:'Status', data.columns[1]:'time'})
    data2 = data[data['time'].astype(float) >0]
    data2 = data2.apply(pd.to_numeric,errors='ignore')

    if type(data2['Status'].unique()[0]) == str :
        data2['Status'] = data2['Status'].str.lower()
        x,y = get_x_y(data2,data2.columns[:2],pos_label='dead')
    else:
        x,y = get_x_y(data2,data2.columns[:2],pos_label=1)
    #转变数据类型，并进行独热编码
    x = x.apply(pd.to_numeric,errors='ignore')
    x = pd.get_dummies(x)
    return x,y